In [9]:
import pandas as pd
from lib import standardize_item_numbers
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go


def clean():
    dfa = pd.read_csv("../data/police_reports/electronic_police_report_2018_2022.csv")
    dfa = dfa.pipe(standardize_item_numbers, ["item_number"])

    dfb = pd.read_csv("../data/real_time_crime_center/rtcc.csv")
    dfb["footage_requested"] = "1"
    dfb = dfb.pipe(standardize_item_numbers, ["item_number"])

    df = pd.merge(dfa, dfb, on="item_number")
    return df, dfa

In [10]:
rtcc_requested_df, no_rtcc_df = clean()
rtcc_requested_df_og = rtcc_requested_df.copy()

C:\Users\PC\AppData\Local\Temp\ipykernel_20808\3509277380.py:9: DtypeWarning:

Columns (13,23,24) have mixed types. Specify dtype option on import or set low_memory=False.



In [11]:
no_rtcc_df.loc[:, "occurred_year"] = no_rtcc_df.occurred_date_time.str.replace(r".+\/(\w{4}) .+", r"\1")
no_rtcc_df = no_rtcc_df[no_rtcc_df.charge_description.isin(["FIRST DEGREE MURDER", "SECOND DEGREE MURDER", "AGG. BATTERY", "SECOND DEGREE BATTERY",
                                    "AGG. ASSAULT", "FIRST DEGREE RAPE", "SECOND DEGREE RAPE", "THIRD DEGREE RAPE",
                                    "SEXUAL BATTERY", "SIMPLE KIDNAPPING", "AGG. KIDNAPPING",
                                    "AGG. ARSON", "CRIMINAL DAMAGE TO PROPERTY",
                                    "AGG. BURGLARY", "FIRST DEGREE ROBBERY", "ARMED ROBBERY",
                                    "SIMPLE ROBBERY", "ASSAULT BY DRIVE BY SHOOTING",
                                    "CARJACKING", "TERRORIZING", "AGG. SECOND DEGREE BATTERY",
                                    "AGGRAVATED ASSAULT UPON A PEACE OFFICER", "AGGRAVATED ASSAULT WITH A FIREARM",
                                    "SECOND DEGREE ROBBERY", "2ND DEGREE CRUELTY TO JUVENILES",
                                    "BATTERY OF A POLICE OFFICER", "RELATIVE TO HOME INVASION"])]
no_rtcc_df.occurred_year.value_counts()

C:\Users\PC\AppData\Local\Temp\ipykernel_20808\3890351779.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



2021    4520
2022    4379
2020    3969
2018    3905
2019    3568
Name: occurred_year, dtype: int64

In [12]:
rtcc_requested_df = rtcc_requested_df[rtcc_requested_df.charge_description.isin(["FIRST DEGREE MURDER", "SECOND DEGREE MURDER", "AGG. BATTERY", "SECOND DEGREE BATTERY",
                                    "AGG. ASSAULT", "FIRST DEGREE RAPE", "SECOND DEGREE RAPE", "THIRD DEGREE RAPE",
                                    "SEXUAL BATTERY", "SIMPLE KIDNAPPING", "AGG. KIDNAPPING",
                                    "AGG. ARSON", "CRIMINAL DAMAGE TO PROPERTY",
                                    "AGG. BURGLARY", "FIRST DEGREE ROBBERY", "ARMED ROBBERY",
                                    "SIMPLE ROBBERY", "ASSAULT BY DRIVE BY SHOOTING",
                                    "CARJACKING", "TERRORIZING", "AGG. SECOND DEGREE BATTERY",
                                    "AGGRAVATED ASSAULT UPON A PEACE OFFICER", "AGGRAVATED ASSAULT WITH A FIREARM",
                                    "SECOND DEGREE ROBBERY", "2ND DEGREE CRUELTY TO JUVENILES",
                                    "BATTERY OF A POLICE OFFICER", "RELATIVE TO HOME INVASION"])]

rtcc_requested_df.loc[:, "occurred_year"] = rtcc_requested_df.occurred_date_time.str.replace(r".+\/(\w{4}) .+", r"\1")
rtcc_requested_df = pd.DataFrame(rtcc_requested_df.groupby("occurred_year").disposition.value_counts())
rtcc_requested_df  = rtcc_requested_df.rename(columns={rtcc_requested_df.columns[0]: "rtcc_requested_count"})
rtcc_requested_df = rtcc_requested_df.reset_index()

rtcc_requested_df_open = rtcc_requested_df[rtcc_requested_df.disposition.isin(["OPEN"])]

rtcc_requested_df_open.loc[:, "occurred_year"]= np.array(rtcc_requested_df_open.occurred_year, dtype=float)
rtcc_requested_df_open = rtcc_requested_df_open.sort_values("occurred_year").reset_index(drop=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_20808\1605641364.py:12: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\PC\AppData\Local\Temp\ipykernel_20808\1605641364.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
rtcc_requested_item_nos = [num for num in rtcc_requested_df_og["item_number"]]

no_rtcc_df = no_rtcc_df[~((no_rtcc_df.item_number.isin(rtcc_requested_item_nos)))]
no_rtcc_df.loc[:, "occurred_year"] = no_rtcc_df.occurred_date_time.str.replace(r".+\/(\w{4}) .+", r"\1")

no_rtcc_df = no_rtcc_df[no_rtcc_df.charge_description.isin(["FIRST DEGREE MURDER", "SECOND DEGREE MURDER", "AGG. BATTERY", "SECOND DEGREE BATTERY",
                                    "AGG. ASSAULT", "FIRST DEGREE RAPE", "SECOND DEGREE RAPE", "THIRD DEGREE RAPE",
                                    "SEXUAL BATTERY", "SIMPLE KIDNAPPING", "AGG. KIDNAPPING",
                                    "AGG. ARSON", "CRIMINAL DAMAGE TO PROPERTY",
                                    "AGG. BURGLARY", "FIRST DEGREE ROBBERY", "ARMED ROBBERY",
                                    "SIMPLE ROBBERY", "ASSAULT BY DRIVE BY SHOOTING",
                                    "CARJACKING", "TERRORIZING", "AGG. SECOND DEGREE BATTERY",
                                    "AGGRAVATED ASSAULT UPON A PEACE OFFICER", "AGGRAVATED ASSAULT WITH A FIREARM",
                                    "SECOND DEGREE ROBBERY", "2ND DEGREE CRUELTY TO JUVENILES",
                                    "BATTERY OF A POLICE OFFICER", "RELATIVE TO HOME INVASION"])]

no_rtcc_df = pd.DataFrame(no_rtcc_df.groupby("occurred_year").disposition.value_counts())
no_rtcc_df  = no_rtcc_df.rename(columns={no_rtcc_df.columns[0]: "rtcc_not_requested_count"})
no_rtcc_df = no_rtcc_df.reset_index()

no_rtcc_df_open = no_rtcc_df[no_rtcc_df.disposition.isin(["OPEN"])]

no_rtcc_df_open.loc[:, "occurred_year"]= np.array(no_rtcc_df_open.occurred_year, dtype=float)
no_rtcc_df_open = no_rtcc_df_open.sort_values("occurred_year").reset_index(drop=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_20808\2395727109.py:4: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\PC\AppData\Local\Temp\ipykernel_20808\2395727109.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
fig = go.Figure()

fig.add_scatter(x=no_rtcc_df_open['occurred_year'], y=no_rtcc_df_open['rtcc_not_requested_count'], mode="lines+text",
    name="Open Violent Crime Cases without RTCC Video", )
fig.add_scatter(x=rtcc_requested_df_open['occurred_year'], y=rtcc_requested_df_open['rtcc_requested_count'], mode="lines+text",
    name="Open Violent Crime Cases with RTCC Video",)

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [2018, 2019, 2020, 2021, 2022],
    )
)

fig.update_layout(legend=dict(
    yanchor="bottom",
    y=-0.5,
    xanchor="center",
    x=0.5
))

fig.update_layout(
    xaxis_title="Year", yaxis_title="Number of Violent Crimes",
    title = "Open violent crime cases"
)

fig.show()